In [2]:
import pandas as pd
import dapla as dp
import qgrid
import numpy as np

## Relativt elegant, men kompleks datainnlast

In [3]:
#dp.show('/felles/mock_sysselsatte')

In [4]:
# Man kan skrive inn understreker i tall, for bedre lesbarhet, men de vil senere bli ignorert.
pop_size = 10_000
# Siste årstall, skal være siste årstall+1
aar_serie = (2019, 2023 + 1)

In [5]:
# List-comprehensions for å generere alle paths, med tuple unpacking for årstall
pop_in_paths = [f'/felles/mock_sysselsatte/population_{i}_{pop_size}' for i in range( *aar_serie )]
comp_in_paths = [f'/felles/mock_sysselsatte/companies_{i}_{pop_size}' for i in range( *aar_serie )]

In [6]:
# Dictionary som dataframene skal stappes inn i
dfs = {}
# Loop gjennom alle pathene vi lagde med comprehensionene
for in_path in pop_in_paths + comp_in_paths:
    print('Laster inn', in_path)
    # Lag keys fra pathene, og les inn dataframes fra daplalagringen på de dem
    dfs['_'.join(in_path.split('/')[-1].split('_')[:2])] = dp.read_pandas(in_path)

Laster inn /felles/mock_sysselsatte/population_2019_10000
Laster inn /felles/mock_sysselsatte/population_2020_10000
Laster inn /felles/mock_sysselsatte/population_2021_10000
Laster inn /felles/mock_sysselsatte/population_2022_10000
Laster inn /felles/mock_sysselsatte/population_2023_10000
Laster inn /felles/mock_sysselsatte/companies_2019_10000
Laster inn /felles/mock_sysselsatte/companies_2020_10000
Laster inn /felles/mock_sysselsatte/companies_2021_10000
Laster inn /felles/mock_sysselsatte/companies_2022_10000
Laster inn /felles/mock_sysselsatte/companies_2023_10000


In [7]:
# Eksempler på henting av data ut fra dictionaryen
comp2020 = dfs['companies_2020'].drop('employee_points', axis = 'columns')
comp2020.tail(5)

,work_id,nace,region_code,region
396,328824379,11.060,03,Oslo
397,239397238,10.120,02,Akershus (-2019)
398,868069954,11.060,03,Oslo
399,492650911,46.390,11,Rogaland
400,573146486,11.010,03,Oslo


In [8]:
pop2020 = dfs['population_2020']
pop2020

,id,work_id,sex,age,year_birth,work_percent
0,09031988152,None,Menn,1.0,2019.0,NaN
1,20121929940,None,Menn,1.0,2019.0,NaN
2,02041986972,None,Menn,1.0,2019.0,NaN
3,20081951802,None,Menn,1.0,2019.0,NaN
4,20101905788,None,Menn,1.0,2019.0,NaN
...,...,...,...,...,...,...
10066,10087793930,None,Kvinner,0.0,1977.0,NaN
10067,23017098997,None,Kvinner,0.0,1970.0,NaN
10068,18057184737,None,Kvinner,0.0,1971.0,NaN
10069,21056625708,None,Kvinner,0.0,1966.0,NaN


## Koble (merge) data fra selskaper på personer

In [9]:
popcomp2020 = pop2020.merge(comp2020, 
                            how = 'left', 
                            on = 'work_id' )
popcomp2020

,id,work_id,sex,age,year_birth,work_percent,nace,region_code,region
0,09031988152,None,Menn,1.0,2019.0,NaN,NaN,NaN,NaN
1,20121929940,None,Menn,1.0,2019.0,NaN,NaN,NaN,NaN
2,02041986972,None,Menn,1.0,2019.0,NaN,NaN,NaN,NaN
3,20081951802,None,Menn,1.0,2019.0,NaN,NaN,NaN,NaN
4,20101905788,None,Menn,1.0,2019.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10066,10087793930,None,Kvinner,0.0,1977.0,NaN,NaN,NaN,NaN
10067,23017098997,None,Kvinner,0.0,1970.0,NaN,NaN,NaN,NaN
10068,18057184737,None,Kvinner,0.0,1971.0,NaN,NaN,NaN,NaN
10069,21056625708,None,Kvinner,0.0,1966.0,NaN,NaN,NaN,NaN


In [10]:
# Personer registrert med arbeidssted
popcomp2020[~popcomp2020['work_id'].isnull()].head()

,id,work_id,sex,age,year_birth,work_percent,nace,region_code,region
798,21120465166,482770609,Menn,16.0,2004.0,100.0,10.850,03,Oslo
802,21070494908,962204561,Menn,16.0,2004.0,100.0,47.524,14,Sogn og Fjordane (-2019)
807,18080483122,030786943,Menn,16.0,2004.0,100.0,30.400,08,Telemark (-2019)
833,08050483952,880841018,Menn,16.0,2004.0,53.0,47.990,50,Trøndelag - Trööndelage
840,04050476852,280367594,Menn,16.0,2004.0,35.0,01.630,01,Østfold (-2019)


## Start Qgrid

In [11]:
qgrid_widget = qgrid.show_grid(popcomp2020, show_toolbar = True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
df_selection = qgrid_widget.get_changed_df()
df_selection

,id,work_id,sex,age,year_birth,work_percent,nace,region_code,region
4211,12017349948,296127052,Menn,47.0,1973.0,99.0,01.420,01,Østfold (-2019)
2772,31087603580,890987163,Menn,44.0,1976.0,85.0,02.300,01,Østfold (-2019)
2566,23037747004,177326491,Menn,43.0,1977.0,100.0,02.100,01,Østfold (-2019)
3082,17096855658,101919785,Menn,52.0,1968.0,100.0,01.460,01,Østfold (-2019)
1417,23097663422,912099265,Menn,44.0,1976.0,100.0,01.280,01,Østfold (-2019)
...,...,...,...,...,...,...,...,...,...
3830,30106818178,182129826,Menn,52.0,1968.0,100.0,52.211,50,Trøndelag - Trööndelage
1728,27017655182,666156439,Menn,44.0,1976.0,26.0,50.300,50,Trøndelag - Trööndelage
3456,07066804038,572389326,Menn,52.0,1968.0,100.0,52.214,50,Trøndelag - Trööndelage
4050,16107311914,870025975,Menn,47.0,1973.0,100.0,50.102,50,Trøndelag - Trööndelage


In [13]:
# After upgrading to 1.0.3
def handle_json_updated(event, qgrid_widget):
    # exclude 'viewport_changed' events since that doesn't change the DataFrame
    if (event['triggered_by'] != 'viewport_changed'):
        print(qgrid_widget.get_changed_df())

qgrid_widget.on('json_updated', handle_json_updated)

In [14]:
df_selection = qgrid_widget.get_changed_df()
row_ids = df_selection.index.tolist()

### Her bør du logge endringene du gjør ett eller annet sted...

In [15]:
# Dette er sånt som hadde vært smart å logget, før man påførte det på datasettet
diff = popcomp2020.loc[row_ids].compare(df_selection)
diff

work_percent       
             self  other
1900         51.0  100.0
3982         75.0  100.0

In [ ]:
# Navn på datasett endret, brukernavn, tidspunkt, endringskommentar

### Du kan overskrive dataene i det originale datasettet, men hva gjør du da for å endre det tilbake?

In [110]:
# Skriv over rader i orginaldataframe med endringene
for i, row in df_selection.iterrows():
    popcomp2020.loc[i] = row

In [111]:
popcomp2020.loc[row_ids]

,id,work_id,sex,age,year_birth,work_percent,nace,region_code,region
4089,19116806296,919861046,Menn,53.0,1968.0,NaN,01.140,01,Østfold (-2019)
3982,21027405926,919861046,Menn,46.0,1974.0,75.0,01.140,01,Østfold (-2019)
1900,25017941998,919861046,Menn,41.0,1979.0,52.0,01.140,01,Østfold (-2019)
